In [ ]:
dataset_train = pd.read_csv('/content/trainset.csv')

In [ ]:
dataset_train.columns


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras import layers
from keras.models import Sequential

In [ ]:
dataset_train.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,02-01-2013,357.385559,361.151062,355.959839,359.288177,359.288177,5115500
1,03-01-2013,360.122742,363.600128,358.031342,359.496826,359.496826,4666500
2,04-01-2013,362.313507,368.339294,361.488861,366.600616,366.600616,5562800
3,07-01-2013,365.348755,367.301056,362.929504,365.001007,365.001007,3332900
4,08-01-2013,365.393463,365.771027,359.874359,364.280701,364.280701,3373900


In [ ]:
train_set = dataset_train.iloc[:,1:2].values


In [ ]:
type(train_set)


numpy.ndarray

In [ ]:
train_set.shape


(1259, 1)

In [ ]:

sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(train_set)

In [ ]:
training_set_scaled.shape


(1259, 1)

In [ ]:
X_train_array = []
y_train_array = []
for i in range(60, 1259):
  X_train_array.append(training_set_scaled[i-60:i,0])
  y_train_array.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train_array), np.array(y_train_array)
X_train1 = X_train.reshape((X_train.shape[0], X_train.shape[1],1))

In [ ]:
X_train.shape


(1199, 60)

In [ ]:
length = 60
n_features = 1


In [ ]:
model = Sequential()
model.add(layers.SimpleRNN(50,input_shape=(60,1)))
model.add(layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 50)                2600      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2651 (10.36 KB)
Trainable params: 2651 (10.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train1,y_train,epochs=100, batch_size=32)

Epoch 1/100
38/38 [==============================] - 7s 34ms/step - loss: 0.0095
Epoch 2/100
38/38 [==============================] - 2s 58ms/step - loss: 5.7368e-04
Epoch 3/100
38/38 [==============================] - 2s 43ms/step - loss: 4.4970e-04
Epoch 4/100
38/38 [==============================] - 1s 34ms/step - loss: 4.2233e-04
Epoch 5/100
38/38 [==============================] - 1s 37ms/step - loss: 3.9264e-04
Epoch 6/100
38/38 [==============================] - 1s 36ms/step - loss: 3.9376e-04
Epoch 7/100
38/38 [==============================] - 1s 34ms/step - loss: 3.8173e-04
Epoch 8/100
38/38 [==============================] - 1s 35ms/step - loss: 3.3657e-04
Epoch 9/100
38/38 [==============================] - 1s 36ms/step - loss: 3.5257e-04
Epoch 10/100
38/38 [==============================] - 3s 87ms/step - loss: 3.2161e-04
Epoch 11/100
38/38 [==============================] - 2s 47ms/step - loss: 3.1223e-04
Epoch 12/100
38/38 [==============================] - 1s 36ms/step 

In [ ]:
dataset_test = pd.read_csv('testset.csv')


In [ ]:
test_set = dataset_test.iloc[:,1:2].values


In [ ]:
test_set.shape


(125, 1)

In [ ]:
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']),axis=0)


In [ ]:
inputs = dataset_total.values
inputs = inputs.reshape(-1,1)
inputs_scaled=sc.transform(inputs)
X_test = []
for i in range(60,1384):
  X_test.append(inputs_scaled[i-60:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))

In [ ]:
X_test.shape


(1324, 60, 1)

In [ ]:
predicted_stock_price_scaled = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price_scaled)

42/42 [==============================] - 0s 6ms/step


In [ ]:
plt.plot(np.arange(0,1384),inputs, color='blue', label = 'Test(Real) Google stock price')
plt.plot(np.arange(60,1384),predicted_stock_price, color='red', label = 'Predicted Google stock price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()